In [0]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F

In [0]:
wine_data = pd.read_csv('/content/drive/My Drive/Deep_Learning_Projects/Wine_Dataset/wine.csv')

In [26]:
wine_data.head()

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,0,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,0,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,0,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,0,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [27]:
wine_data.shape

(178, 14)

In [28]:
wine_data.sample(8)

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
167,2,12.82,3.37,2.30,19.5,88,1.48,0.66,0.40,0.97,10.26,0.72,1.75,685
27,0,13.30,1.72,2.14,17.0,94,2.40,2.19,0.27,1.35,3.95,1.02,2.77,1285
86,1,12.16,1.61,2.31,22.8,90,1.78,1.69,0.43,1.56,2.45,1.33,2.26,495
56,0,14.22,1.70,2.30,16.3,118,3.20,3.00,0.26,2.03,6.38,0.94,3.31,970
154,2,12.58,1.29,2.10,20.0,103,1.48,0.58,0.53,1.40,7.60,0.58,1.55,640
100,1,12.08,2.08,1.70,17.5,97,2.23,2.17,0.26,1.40,3.30,1.27,2.96,710
168,2,13.58,2.58,2.69,24.5,105,1.55,0.84,0.39,1.54,8.66,0.74,1.80,750
135,2,12.60,2.46,2.20,18.5,94,1.62,0.66,0.63,0.94,7.10,0.73,1.58,695


In [29]:
wine_features = wine_data.drop('Class', axis=1)
wine_features.sample(8)

,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
142,13.52,3.17,2.72,23.5,97,1.55,0.52,0.50,0.55,4.35,0.89,2.06,520
48,14.10,2.02,2.40,18.8,103,2.75,2.92,0.32,2.38,6.20,1.07,2.75,1060
145,13.16,3.57,2.15,21.0,102,1.50,0.55,0.43,1.30,4.00,0.60,1.68,830
5,14.20,1.76,2.45,15.2,112,3.27,3.39,0.34,1.97,6.75,1.05,2.85,1450
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
29,14.02,1.68,2.21,16.0,96,2.65,2.33,0.26,1.98,4.70,1.04,3.59,1035
18,14.19,1.59,2.48,16.5,108,3.30,3.93,0.32,1.86,8.70,1.23,2.82,1680
72,13.49,1.66,2.24,24.0,87,1.88,1.84,0.27,1.03,3.74,0.98,2.78,472


In [30]:
wine_target = wine_data[['Class']]
wine_target.sample(5)

,Class
81,1
91,1
1,0
98,1
128,1


In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(wine_features, wine_target, test_size=0.4, random_state=0)

In [32]:
X_train.shape, Y_train.shape

((106, 13), (106, 1))

In [0]:
X_train = torch.from_numpy(X_train.values).float()
X_test = torch.from_numpy(X_test.values).float()

In [34]:
X_train.shape, X_test.shape

(torch.Size([106, 13]), torch.Size([72, 13]))

In [0]:
Y_train = torch.from_numpy(Y_train.values).view(1,-1)[0]
Y_test = torch.from_numpy(Y_test.values).view(1,-1)[0]

In [36]:
Y_train.shape, Y_test.shape

(torch.Size([106]), torch.Size([72]))

In [0]:
input_size = 13
output_size = 3
hidden_size = 100

In [0]:
class Net(nn.Module):

  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.fc2 = nn.Linear(hidden_size, hidden_size)
    self.fc3 = nn.Linear(hidden_size, output_size)

  def forward(self, X):
    X = torch.sigmoid(self.fc1(X))
    X = torch.sigmoid(self.fc2(X))
    X = self.fc3(X)

    return F.log_softmax(X, dim=1)

In [0]:
model = Net()

In [0]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr = 0.01)

loss_fn = nn.NLLLoss()

In [0]:
epochs = 100

In [42]:
for epoch in range(epochs):

  optimizer.zero_grad()
  Y_pred = model(X_train)

  loss = loss_fn(Y_pred, Y_train)
  loss.backward()

  optimizer.step()

  if epoch%10 == 0:
    print('Epoch : ', epoch, ' Loss : ', loss.item())

Epoch :  0  Loss :  1.0997155904769897
Epoch :  10  Loss :  1.0037118196487427
Epoch :  20  Loss :  0.750158965587616
Epoch :  30  Loss :  0.6580314040184021
Epoch :  40  Loss :  0.6314710974693298
Epoch :  50  Loss :  0.6031659245491028
Epoch :  60  Loss :  0.5804309844970703
Epoch :  70  Loss :  0.5723595023155212
Epoch :  80  Loss :  0.5465887784957886
Epoch :  90  Loss :  0.5378631353378296


In [43]:
cd '/content/drive/My Drive/Deep_Learning_Projects/Wine_Dataset/'

/content/drive/My Drive/Deep_Learning_Projects/Wine_Dataset


In [0]:
torch.save({
    'epoch' : epoch,
    'model_state_dict' : model.state_dict(),
    'optimizer_state_dict' : optimizer.state_dict(),
    'loss' : loss, 
}, 'models/wine_checkpoint.pth')

In [0]:
predict_out = model(X_test)
_, predict_y = torch.max(predict_out, 1)

In [0]:
from sklearn.metrics import precision_score, accuracy_score, recall_score

In [47]:
print("Accuracy score : ", accuracy_score(Y_test.data, predict_y.data))
print("Precision score : ", precision_score(Y_test.data, predict_y.data, average='micro'))
print("Recall score : ", recall_score(Y_test.data, predict_y.data, average='micro'))

Accuracy score :  0.6944444444444444
Precision score :  0.6944444444444444
Recall score :  0.6944444444444444
